# Quantum Key Distribution

Quantum key distribution is the process of distributing cryptographic keys between parties using quantum methods. Due to the unique properties of quantum information compared to classical, the security of a key can be guarunteed (as any unwelcomed measurement would change the state of quantum information transmitted).

Experimental setup for QKD is as follows:

<img src="./notebook_images/qkd.png" width="500"/>

##  QKD without cascade protocol

- `Timeline` provides an interface for the discrete-event simulation kernel.
- `QKDNode` implements quantum key distribution, including necessary hardware and protocol implementations.
- `QuantumChannel` and `ClassicalChannel` serve as communication links between the nodes.
- The `pair_bb84_protocols` function is used to explicitly pair 2 node instances for key distribution, and establishes one node as the sender "Alice" and one as the receiver "Bob".

In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [2]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QKDNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.qkd.BB84 import pair_bb84_protocols

ModuleNotFoundError: No module named 'sequence'

In [3]:
class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) 
            
    def pop(self, info): 
        self.keys.append(info)
        self.times.append(self.timeline.now() * 1e-9)

In [4]:
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    """
    tl = Timeline(sim_time * 1e9)
    tl.seed(0)
    
    n1 = QKDNode("n1", tl, stack_size=1)
    n2 = QKDNode("n2", tl, stack_size=1)
    pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])
    

    cc0 = ClassicalChannel("cc_n1_n2", tl, distance=1e3)
    cc1 = ClassicalChannel("cc_n2_n1", tl, distance=1e3)
    cc0.set_ends(n1, n2.name)
    cc1.set_ends(n2, n1.name)
    qc0 = QuantumChannel("qc_n1_n2", tl, attenuation=1e-5, distance=1e3, polarization_fidelity=0.97)
    qc1 = QuantumChannel("qc_n2_n1", tl, attenuation=1e-5, distance=1e3, polarization_fidelity=0.97)
    qc0.set_ends(n1, n2.name)
    qc1.set_ends(n2, n1.name)
    
    km1 = KeyManager(tl, keysize, 25)
    km1.lower_protocols.append(n1.protocol_stack[0])
    n1.protocol_stack[0].upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 25)
    km2.lower_protocols.append(n2.protocol_stack[0])
    n2.protocol_stack[0].upper_protocols.append(km2)
    
    tl.init()
    km1.send_request()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.xlabel("Simulation time (ms)")
    plt.ylabel("Number of Completed Keys")
    plt.show()
    
    print("key error rates:")
    for i, e in enumerate(n1.protocol_stack[0].error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

### Execution

Parameters:

    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)

In [5]:
# Create and run the simulation
interactive_plot = interact(test, sim_time=(100, 1000, 100), keysize=[128, 256, 512])
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), Dropdown(desc…

<function __main__.test(sim_time, keysize)>

## QKD with cascade protocol


In [6]:
from sequence.qkd.cascade import pair_cascade_protocols

class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) 
            
    def pop(self, key): 
        self.keys.append(key)
        self.times.append(self.timeline.now() * 1e-9)
        
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    """
    tl = Timeline(sim_time * 1e9)
    tl.seed(0)
    
    n1 = QKDNode("n1", tl)
    n2 = QKDNode("n2", tl)
    pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])
    pair_cascade_protocols(n1.protocol_stack[1], n2.protocol_stack[1])
    
    cc0 = ClassicalChannel("cc_n1_n2", tl, distance=1e3)
    cc1 = ClassicalChannel("cc_n2_n1", tl, distance=1e3)
    cc0.set_ends(n1, n2.name)
    cc1.set_ends(n2, n1.name)
    qc0 = QuantumChannel("qc_n1_n2", tl, attenuation=1e-5, distance=1e3, polarization_fidelity=0.97)
    qc1 = QuantumChannel("qc_n2_n1", tl, attenuation=1e-5, distance=1e3, polarization_fidelity=0.97)
    qc0.set_ends(n1, n2.name)
    qc1.set_ends(n2, n1.name)
    
    km1 = KeyManager(tl, keysize, 10)
    km1.lower_protocols.append(n1.protocol_stack[1])
    n1.protocol_stack[1].upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 10)
    km2.lower_protocols.append(n2.protocol_stack[1])
    n2.protocol_stack[1].upper_protocols.append(km2)
    
    tl.init()
    km1.send_request()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.xlabel("Simulation time (ms)")
    plt.ylabel("Number of Completed Keys")
    plt.show()
    
    error_rates = []
    for i, key in enumerate(km1.keys):
        counter = 0
        diff = key ^ km2.keys[i]
        for j in range(km1.keysize):
            counter += (diff >> j) & 1
        error_rates.append(counter)

    print("key error rates:")
    for i, e in enumerate(error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

### Execution

Parameters:

    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    

In [7]:
interactive_plot = interact(test, sim_time=(100, 1000, 100), keysize=[128, 256, 512])
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), Dropdown(desc…

<function __main__.test(sim_time, keysize)>